In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

URL = "https://mamaearth.in/product-category/makeup-products"
HEADERS = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36','Accept-Language': 'en-US,en;q=0.5'}

In [10]:
webpage = requests.get(URL,headers=HEADERS)

if (webpage.status_code == 200 ):
    print("Data Fetched Successfully !")
else:
    print(f"Error{webpage.status_code} !")

soup = BeautifulSoup(webpage.content,'html.parser')


Data Fetched Successfully !


In [37]:
# Extracting product links

product_links = soup.find_all("div",attrs={'class':'ProductCard_Wrapper_DisplayArea'})
# for links_tag in links:
# product_link = "https://mamaearth.in" + links[0].find("a").get('href')

links_list = []
for link in product_links:
    product_link =  "https://mamaearth.in" + link.find("a").get('href')
    links_list.append(product_link)

scrap_data = {'Product_Name':[],'Description':[],'Price':[],'Rating out of 5':[]}


In [13]:
# Function to extract product details:

#Function to extract product name

def fetch_productName(new_soup):
    
    try:
        product_name = new_soup.find("h1",attrs={'class':'ProductDetailsRevamp__ProdName-sc-1w9tx2u-2 hrSRwG'}).text.strip()
    except AttributeError:
        product_name = "Not Available"
        
    return product_name

# Function to extract product description

def fetch_productDescription(new_soup):
    
    try:
        product_description = new_soup.find("div",attrs={'class':'subtitle'}).text.strip()
    except AttributeError:
        product_description = "NIL"
        
    return product_description

# Function to extract product price

def fetch_productPrice(new_soup):
    
    try:
        product_price = new_soup.find("td",attrs={'class':'price'}).text.strip()
    except AttributeError:
        product_price = "Not Available"
        
    return product_price

#Function to extract product Rating

def fetch_productRating(new_soup):
    
    try:
        rating_tag = new_soup.find("a",attrs={'class':'rating'})
        rating_soup = BeautifulSoup(rating_tag.prettify())
        
        span_tags = rating_soup.find_all("span")
        product_rating = span_tags[0].text.strip()+" "+span_tags[1].text.strip()
    except AttributeError:
        product_rating = "NA"
        
    return product_rating
        
    

In [27]:
# Looping through all the links and scraping data :

for link in links_list:
    new_webpage = requests.get(link,headers=HEADERS)
    
    if (new_webpage.status_code != 200 ):
        continue
    new_soup = BeautifulSoup(new_webpage.content,'html.parser')
    
    scrap_data['Product_Name'].append(fetch_productName(new_soup))
    scrap_data['Description'].append(fetch_productDescription(new_soup))
    scrap_data['Price'].append(fetch_productPrice(new_soup))
    scrap_data['Rating out of 5'].append(fetch_productRating(new_soup))
    

In [38]:
# Generating a dataframe using pandas library

mamaEarth_df = pd.DataFrame.from_dict(scrap_data)

# Converting the scrap data to a CSV File

mamaEarth_df.to_csv("mamaEarth_scrapData.csv",index=True)

In [ ]:
mamaEarth_df